In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openpyxl
# regresion de categorías con scikit-learn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# one hot encoding
from sklearn.preprocessing import OneHotEncoder
# para el escalado de datos
from sklearn.preprocessing import StandardScaler


In [42]:
# cargar los datos que están en la carpeta anterior a esta
# df = pd.read_csv('./BD Prueba técnica Analista de datos.xlsx')
#carga el archivo xlsx
df = pd.read_excel('BD Prueba técnica Analista de datos.xlsx')


In [43]:

df.head()


,No.,SalePrice,YearBuilt,YrSold,MonthSold,Size.sqf.,Floor,HallwayType,HeatingType,AptManageType,...,N_FacilitiesNearBy.Mall.,N_FacilitiesNearBy.ETC.,N_FacilitiesNearBy.Park.,N_SchoolNearBy.Elementary.,N_SchoolNearBy.Middle.,N_SchoolNearBy.High.,N_SchoolNearBy.University.,N_FacilitiesInApt,N_FacilitiesNearBy.Total.,N_SchoolNearBy.Total.
0,1,141592,2006,2007,8,814,3,terraced,individual_heating,management_in_trust,...,1,1,0,3,2,2,2,5,6,9
1,2,51327,1985,2007,8,587,8,corridor,individual_heating,self_management,...,1,2,1,2,1,1,0,3,12,4
2,3,48672,1985,2007,8,587,6,corridor,individual_heating,self_management,...,1,2,1,2,1,1,0,3,12,4
3,5,221238,1993,2007,8,1761,3,mixed,individual_heating,management_in_trust,...,1,5,0,4,3,5,5,4,14,17
4,7,78318,1992,2007,8,644,2,mixed,individual_heating,self_management,...,1,1,0,3,3,4,4,3,9,14


In [44]:
# Hay 31 columnas. quitar desde la columna 21 hasta la 29
df = df.drop(df.columns[23:29], axis=1)
df.head()

,No.,SalePrice,YearBuilt,YrSold,MonthSold,Size.sqf.,Floor,HallwayType,HeatingType,AptManageType,...,N_manager,N_elevators,SubwayStation,N_FacilitiesNearBy.PublicOffice.,N_FacilitiesNearBy.Hospital.,N_FacilitiesNearBy.Dpartmentstore.,N_FacilitiesNearBy.Mall.,N_FacilitiesNearBy.ETC.,N_FacilitiesNearBy.Total.,N_SchoolNearBy.Total.
0,1,141592,2006,2007,8,814,3,terraced,individual_heating,management_in_trust,...,3,0,Kyungbuk_uni_hospital,2,1,1,1,1,6,9
1,2,51327,1985,2007,8,587,8,corridor,individual_heating,self_management,...,2,2,Daegu,5,1,2,1,2,12,4
2,3,48672,1985,2007,8,587,6,corridor,individual_heating,self_management,...,2,2,Daegu,5,1,2,1,2,12,4
3,5,221238,1993,2007,8,1761,3,mixed,individual_heating,management_in_trust,...,8,20,Myung-duk,6,2,0,1,5,14,17
4,7,78318,1992,2007,8,644,2,mixed,individual_heating,self_management,...,4,8,Myung-duk,5,1,1,1,1,9,14


In [40]:
# hacer one hot encoding de las variables 'HallwayType' 'HeatingType' 'AptManageType' 'TimeToBusStop' 'TimeToSubway'
# Inicializar el OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Ajustar y transformar los datos
encoded_data = encoder.fit_transform(df[['HallwayType', 'HeatingType', 'AptManageType', 'TimeToBusStop', 'TimeToSubway']])

# Obtener nombres de las columnas
encoded_columns = encoder.get_feature_names_out(['HallwayType', 'HeatingType', 'AptManageType', 'TimeToBusStop', 'TimeToSubway'])

# Convertir el resultado a un DataFrame
encoded_df = pd.DataFrame(encoded_data, columns=encoded_columns)

print(encoded_df)

KeyError: "None of [Index(['HallwayType', 'HeatingType', 'AptManageType', 'TimeToBusStop',\n       'TimeToSubway'],\n      dtype='object')] are in the [columns]"

""
0
1
2
3
4
